# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
import pickle
import csv
import numpy

In [ ]:
class Dataset:
    def __init__(self, X, y):
        assert(len(X) == len(y))
        self.X = X
        self.y = y
        self.size = len(X)
        
    def __len__(self):
        return self.size
    
    def to_batch(self, batchsize):
        split_indices = numpy.arange(batchsize, self.size, batchsize)
        Xs = numpy.split(self.X, split_indices)
        ys = numpy.split(self.y, split_indices)
        return [Dataset(X, y) for X, y in zip(Xs, ys)]
    
    def shuffle(self, rand_state=None):
        if rand_state is None:
            indices = numpy.random.permutation(self.size)
        else:
            assert(isinstance(rand_state, numpy.random.RandomState))
            indices = rand_state.permutation(self.size)
        X = self.X[indices]
        y = self.y[indices]
        return Dataset(X, y)
    
class BasicDatasets:
    def __init__(self, train, valid, test):
        assert(isinstance(train, Dataset) and
               isinstance(valid, Dataset) and
               isinstance(test,  Dataset))
        self.train = train
        self.valid = valid
        self.test  = test
        self.__current = None
        
    def __iter__(self):
        return [self.train, self.valid, self.test].__iter__()
    
def extract_dataset_from_pickle(fname):
    with open(fname, mode="rb") as f:
        data = pickle.load(f)
    return Dataset(data["features"], data["labels"])

def load_label_names(fname):
    with open(fname, mode="r") as f:
        reader = csv.reader(f)
        header = next(reader)
        col = header.index("SignName")
        return [row[col] for row in reader]

In [ ]:
pickle_files = [
    "./data/traffic-signs-data/train.p",
    "./data/traffic-signs-data/valid.p",
    "./data/traffic-signs-data/test.p",
]
datasets = [extract_dataset_from_pickle(fname) for fname in pickle_files]
datasets = BasicDatasets(*datasets)
raw_datasets = datasets

label_name_file = "./signnames.csv"
label_names = load_label_names(label_name_file)

def format_label(idx):
    return "{} ({})".format(label_names[idx], idx)

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
def print_dataset_summary(datasets):
    print("Number of training examples =", datasets.train.X.shape[0])
    print("Number of testing examples =", datasets.test.X.shape[0])

    get_image_shape = lambda x: x.shape[1:]
    image_shape = get_image_shape(datasets.test.X)
    assert(image_shape == get_image_shape(datasets.valid.X))
    assert(image_shape == get_image_shape(datasets.test.X))
    print("Image shape =", image_shape)

    get_n_labels = lambda x: numpy.unique(x).size
    n_labels = get_n_labels(datasets.train.y)
    assert(n_labels == get_n_labels(datasets.valid.y))
    assert(n_labels == get_n_labels(datasets.test.y))
    print("Number of labels =", n_labels)

In [ ]:
print_dataset_summary(datasets)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
import matplotlib.pyplot as pyplot
%matplotlib inline

In [ ]:
def plot_each_label_images(dataset, labels, columns=6, rand_seed=114514):
    numpy.random.seed(rand_seed)
    pyplot.subplots_adjust(left=0, right=0.1, bottom=0, top=0.01, wspace=0, hspace=0)
    for label_idx in range(len(labels)):
        print(format_label(label_idx))
        fig = pyplot.figure(figsize=(12,2))
        images = numpy.random.permutation(dataset.X[dataset.y == label_idx])
        for col in range(columns):
            ax = fig.add_subplot(1, columns, col+1, xticks=[], yticks=[])
            ax.imshow(images[col])
        pyplot.show()

In [ ]:
plot_each_label_images(datasets.train, label_names)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
import cv2
from skimage.morphology import disk
from skimage.filters import rank

In [ ]:
def preprocess(images, standardizer=None):
    images = list(map(lambda x: x, images))
    
    def to_gray(image):
        return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    images = [to_gray(image) for image in images]
    
    def local_histogram_equalize(image):
        return rank.equalize(image, selem=disk(30))
    images = [local_histogram_equalize(image) for image in images]
    
    images = numpy.array(images)
    if len(images.shape) == 3:
        s = images.shape
        new_shape = (s[0], s[1], s[2], 1)
        images = images.reshape(new_shape)
        
    if standardizer is None:
        standardizer = Standardizer(images)
        images = standardizer.standardize(images)
        return images, standardizer
    else:
        images = standardizer.standardize(images)
        return images

# Standarize images
# Requirement: zero mean, equal variance
class Standardizer:
    def __init__(self, ref_images):
        dtype = 'float32'
        self.mean = numpy.mean(ref_images).astype(dtype)
        self.std  = numpy.std (ref_images).astype(dtype)

    def standardize(self, images):
        return (images - self.mean) / self.std

In [ ]:
train_X, standardizer = preprocess(raw_datasets.train.X)

valid_X = preprocess(raw_datasets.valid.X, standardizer)
test_X  = preprocess(raw_datasets.test.X, standardizer)

datasets = BasicDatasets(Dataset(train_X, datasets.train.y),
                         Dataset(valid_X, datasets.valid.y),
                         Dataset(test_X,  datasets.test.y))

In [ ]:
print("Mean:", standardizer.mean)
print("Stddev:", standardizer.std)
print()

print("Image pixel type:", datasets.train.X.dtype)
print("Image shape:", datasets.train.X.shape)
print()

print("Train variance:", numpy.var(datasets.train.X))
print("Valid variance:", numpy.var(datasets.valid.X))
print("Test variance:",  numpy.var(datasets.test.X))

### Model Architecture

In [ ]:
import random
import string

import tensorflow
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers import xavier_initializer

In [ ]:
n_labels = len(label_names)

def preferred_networks(x):
    mu = 0
    sigma = 0.1
    
    # Conv: (32x32x1) -> (32x32x32)
    x = ConvolutionLayer(1, (3,3,32), (1,1), 'SAME', mu, sigma).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    
    # ResNet
    #   Conv: (32x32x32) -> (32x32x32)
    #   Activate
    #   Conv: (32x32x32) -> (32x32x32)
    x = Residual([
            ConvolutionLayer(32, (3,3,32), (1,1), 'SAME', mu, sigma),
            ActivationLayer,
            ConvolutionLayer(32, (3,3,32), (1,1), 'SAME', mu, sigma),
            ]).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    # Max pooling: (32x32x32) -> (16x16x32)
    x = MaxPoolingLayer((2,2)).apply(x)

    # Conv: (16x16x32) -> (16x16x128)
    x = ConvolutionLayer(32, (3,3,128), (1,1), 'SAME', mu, sigma).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    
    # ResNet
    #   Conv: (16x16x128) -> (16x16x128)
    #   Activate
    #   Conv: (16x16x128) -> (16x16x128)
    x = Residual([
            ConvolutionLayer(128, (3,3,128), (1,1), 'SAME', mu, sigma),
            ActivationLayer,
            ConvolutionLayer(128, (3,3,128), (1,1), 'SAME', mu, sigma),
            ]).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    # Max pooling: (16x16x128) -> (8x8x128)
    x = MaxPoolingLayer((2,2)).apply(x)
    
    # Conv: (8x8x128) -> (8x8x512)
    x = ConvolutionLayer(128, (3,3,512), (1,1), 'SAME', mu, sigma).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    
    # ResNet
    #   Conv: (8x8x512) -> (8x8x512)
    #   Activate
    #   Conv: (8x8x512) -> (8x8x512)
    x = Residual([
            ConvolutionLayer(512, (3,3,512), (1,1), 'SAME', mu, sigma),
            ActivationLayer,
            ConvolutionLayer(512, (3,3,512), (1,1), 'SAME', mu, sigma),
            ]).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    # Max pooling: (8x8x512) -> (4x4x512)
    x = MaxPoolingLayer((2,2)).apply(x)
    
    # Flatten
    x = flatten(x)
    
    # Fully connected
    x = LinearLayer(8192, 1024, mu, sigma).apply(x)
    # Activate
    x = ActivationLayer.apply(x)
    
    # Fully connected
    x = LinearLayer(1024, n_labels, mu, sigma).apply(x)
    
    return x

class ConvolutionLayer:
    def __init__(self, input_channel, kernel_shape, stride_shape, padding='VALID', mu=0, sigma=0.1):
        assert(padding in ['VALID', 'SAME'])
        def rand_name(n):
            rand_chars = [random.choice(string.ascii_letters + string.digits) for i in range(n)]
            return ''.join(rand_chars)
        # Use xavier initializer for weight
        filter_shape = [kernel_shape[0], kernel_shape[1], input_channel, kernel_shape[2]]
        self.filter = tensorflow.get_variable(
                rand_name(32), shape=filter_shape, initializer=xavier_initializer())
        self.stride = [1, stride_shape[0], stride_shape[1], 1]
        self.bias = tensorflow.Variable(tensorflow.truncated_normal([kernel_shape[2]], mu, sigma))
        self.padding = padding
        
    def apply(self, x):
        x = tensorflow.nn.conv2d(x, self.filter, self.stride, self.padding)
        x = tensorflow.nn.bias_add(x, self.bias)
        return x
    
class MaxPoolingLayer:
    def __init__(self, kernel_shape, stride_shape=None, padding='VALID'):
        assert(padding in ['VALID', 'SAME'])
        self.kernel = [1, kernel_shape[0], kernel_shape[1], 1]
        if stride_shape is None:
            stride_shape = kernel_shape
        self.stride = [1, stride_shape[0], stride_shape[1], 1]
        self.padding = padding
        
    def apply(self, x):
        return tensorflow.nn.max_pool(x, self.kernel, self.stride, self.padding)

class LinearLayer:
    def __init__(self, input_size, output_size, mu=0, sigma=0.1):
        self.weight = tensorflow.Variable(tensorflow.truncated_normal([input_size, output_size], mu, sigma))
        self.bias   = tensorflow.Variable(tensorflow.truncated_normal([output_size], mu, sigma))
    
    def apply(self, x):
        return tensorflow.add(tensorflow.matmul(x, self.weight), self.bias)

class ActivationLayer:
    @classmethod
    def apply(clazz, x):
        return tensorflow.nn.relu(x)
    
class Residual:
    def __init__(self, layers):
        self.layers = layers
        
    def apply(self, x):
        x_in = x
        for layer in self.layers:
            x = layer.apply(x)
        return x + x_in

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
# Hyper parameters
epochs = 10
batchsize = 128
learning_rate = 0.001
rand_seed = 12

In [ ]:
image_h, image_w, image_c = datasets.train.X.shape[1:]

tf_x = tensorflow.placeholder(shape=(None, image_h, image_w, image_c), dtype=tensorflow.float32)
tf_y = tensorflow.placeholder(shape=(None,), dtype=tensorflow.uint8)
tf_onehot_y = tensorflow.one_hot(tf_y, n_labels)
tf_logits = preferred_networks(tf_x)

tf_cross_entropy = tensorflow.nn.softmax_cross_entropy_with_logits_v2(labels=tf_onehot_y, logits=tf_logits)
tf_loss = tensorflow.reduce_mean(tf_cross_entropy)
tf_optimizer = tensorflow.train.AdamOptimizer(learning_rate=learning_rate)
tf_train = tf_optimizer.minimize(tf_loss)

tf_correct = tensorflow.equal(tensorflow.argmax(tf_logits, 1), tensorflow.argmax(tf_onehot_y, 1))
tf_accuracy = tensorflow.reduce_mean(tensorflow.cast(tf_correct, tensorflow.float32))

model_path = "./data/model.tfckpt"
tf_saver = tensorflow.train.Saver()

rand_state = numpy.random.RandomState(rand_seed)

def evaluate(dataset):
    total_accuracy = 0
    sess = tensorflow.get_default_session()
    for batched in dataset.to_batch(batchsize):
        accuracy = sess.run(tf_accuracy,
                            feed_dict={tf_x: batched.X, tf_y: batched.y})
        total_accuracy += (accuracy * len(batched))
    return total_accuracy / len(dataset)

In [ ]:
# Training
with tensorflow.Session() as sess:
    sess.run(tensorflow.global_variables_initializer())
    tensorflow.set_random_seed(rand_seed)
    
    print("Training...")
    print()
    for i in range(epochs):
        batched_list = datasets.train.shuffle(rand_state).to_batch(batchsize)
        for batched in batched_list:
            sess.run(tf_train,
                     feed_dict={tf_x: batched.X, tf_y: batched.y})
            
        valid_accuracy = evaluate(datasets.valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(valid_accuracy))
        print()
        
    tf_saver.save(sess, model_path)
    print("Model saved")

In [ ]:
# Testing
with tensorflow.Session() as sess:
    tf_saver.restore(sess, model_path)
    
    test_accuracy = evaluate(datasets.test)
    print("Test accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
import glob

In [ ]:
def load_image_for_dataset(fname):
    bgr_img = cv2.imread(fname)
    bgr_img = cv2.resize(bgr_img, (32,32))
    return cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)

In [ ]:
new_image_fnames = sorted(list(glob.glob("./data/new_images/*.jpg")))
new_X = [load_image_for_dataset(fname) for fname in new_image_fnames]
new_y = [37, 40, 18, 2, 15]

new_dataset = Dataset(numpy.array(new_X), numpy.array(new_y))
raw_new_dataset = new_dataset
# Preprocess
pp_new_X = preprocess(raw_new_dataset.X, standardizer)
new_dataset = Dataset(pp_new_X, new_dataset.y)

In [ ]:
for image, label_idx in zip(new_X, new_y):
    print(format_label(label_idx))
    pyplot.imshow(image)
    pyplot.show()

In [ ]:
for image, label_idx in zip(pp_new_X, new_y):
    print(format_label(label_idx))
    s = image.shape
    pyplot.imshow(image.reshape([s[0], s[1]]))
    pyplot.show()

### Predict the Sign Type for Each Image

In [ ]:
# Predict
with tensorflow.Session() as sess:
    tf_saver.restore(sess, model_path)
    
    new_accuracy = evaluate(new_dataset)

### Analyze Performance

In [ ]:
print("New accuracy = {:.3f}".format(new_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
with tensorflow.Session() as sess:
    tf_saver.restore(sess, model_path)
    
    top5s = sess.run(tensorflow.nn.top_k(tensorflow.nn.softmax(tf_logits), k=5),
             feed_dict={tf_x: new_dataset.X, tf_y: new_dataset.y})

In [ ]:
for image, correct_label_idx, scores, indices in zip(new_X, new_y, top5s.values, top5s.indices):
    print("====================================")
    pyplot.imshow(image)
    pyplot.show()
    print("Correct: {}".format(format_label(correct_label_idx)))
    for i, (score, index) in enumerate(zip(scores, indices)):
        print("Cand {}:  {} [Score = {}]".format(i, format_label(index), score))

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the pyplot number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    pyplot.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        pyplot.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        pyplot.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            pyplot.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            pyplot.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            pyplot.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            pyplot.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")